In [3]:
import pandas as pd
import re
import os

# Funkcija za učitavanje CSV fajla, filtriranje i dodavanje u zajednički DataFrame
def filtriraj_i_spoji(ulazni_fajl, filter_kljucne_reci, naziv_virusa, svi_proteini):
    # Učitaj CSV fajl
    df = pd.read_csv(ulazni_fajl)
    
    # Kreiranje regularnog izraza koji će odgovarati ključnim rečima u nazivu proteina
    regex_pattern = '|'.join([r'\b' + re.escape(kljucna_rec) + r'\b' for kljucna_rec in filter_kljucne_reci])

    # Filtriraj redove gde naziv proteina sadrži bilo koju od ključnih reči
    filtrirani_df = df[df['Naziv proteina'].str.contains(regex_pattern, case=False, na=False, regex=True)]
    
    # Ako je filtrirani DataFrame prazan, preskoči ga
    if not filtrirani_df.empty:
        # Napravi kopiju filtriranog DataFrame-a
        filtrirani_df_copy = filtrirani_df.copy()
        
        # Dodaj novu kolonu "Virus" sa vrednošću iz naziva fajla
        filtrirani_df_copy['Virus'] = naziv_virusa
        
        # Dodaj filtrirane podatke u zajednički DataFrame
        svi_proteini.append(filtrirani_df_copy)
    else:
        print(f"Nema podataka za ključne reči {filter_kljucne_reci} u fajlu {ulazni_fajl}")

# Funkcija za dobijanje naziva virusa na osnovu naziva fajla
def dobij_naziv_virusa(fajl):
    ime_fajla = os.path.basename(fajl).lower()  # Preuzimamo samo naziv fajla (bez putanje) i pretvaramo u mala slova
    if 'mers' in ime_fajla:
        return 'Mers'
    elif 'marburg' in ime_fajla:
        return 'Marburg'
    elif 'ebola' in ime_fajla:
        return 'Ebola'
    else:
        return 'Nepoznat virus'

# Lista ulaznih fajlova
ulazni_fajlovi = [
    '/home/user/Desktop/Istrazivanje-podataka-2/ISUNSTRUCT_parsiranje/Marburg.csv',
    '/home/user/Desktop/Istrazivanje-podataka-2/ISUNSTRUCT_parsiranje/Mers.csv',
    '/home/user/Desktop/Istrazivanje-podataka-2/ISUNSTRUCT_parsiranje/Ebola.csv'
]

# Ključne reči za različite grupe proteina
kljucne_reci_nucleoprotein = ['___nucleoprotein', '___nucleocapsid']
kljucne_reci_matrix = ['___matrix', '___membrane protein']
kljucne_reci_spike = ['___spike glykoprotein', '___virion spike glycoprotein', '___spike glycoprotein']

# Lista za pohranu svih filtriranih podataka
svi_proteini_nucleoprotein = []
svi_proteini_matrix = []
svi_proteini_spike = []

# Filtriranje podataka za svaki virus i dodavanje u odgovarajuće liste
for fajl in ulazni_fajlovi:
    virus = dobij_naziv_virusa(fajl)
    
    # Filtriraj proteine prema ključnim rečima i spajanje sa svim podacima
    filtriraj_i_spoji(fajl, kljucne_reci_nucleoprotein, virus, svi_proteini_nucleoprotein)
    filtriraj_i_spoji(fajl, kljucne_reci_matrix, virus, svi_proteini_matrix)
    filtriraj_i_spoji(fajl, kljucne_reci_spike, virus, svi_proteini_spike)

# Provera da li je lista svi_proteini_spike prazna
if not svi_proteini_spike:
    print("Nema podataka za proteine spike glykoproteina.")

# Kreiranje DataFrame-a iz filtriranih podataka samo ako postoji nešto za spojiti
if svi_proteini_nucleoprotein:
    df_nucleoprotein = pd.concat(svi_proteini_nucleoprotein, ignore_index=True)
    df_nucleoprotein.to_csv('proteini_nucleoprotein.csv', index=False)

if svi_proteini_matrix:
    df_matrix = pd.concat(svi_proteini_matrix, ignore_index=True)
    df_matrix.to_csv('proteini_matrix_membrane.csv', index=False)

if svi_proteini_spike:
    df_spike = pd.concat(svi_proteini_spike, ignore_index=True)
    df_spike.to_csv('proteini_spike_glykoprotein.csv', index=False)

print("Filtriranje i snimanje je završeno.")

Filtriranje i snimanje je završeno.
